In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho para a raiz do projeto Kedro com base na estrutura de diretórios
# Neste caso, navegamos 4 níveis acima até a raiz do projeto
project_path = os.path.abspath(os.path.join(notebook_cwd, "../../../mtg-project"))

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

[09/14/24 16:25:05] INFO     Using                                                                  ]8;id=836886;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=66710;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


In [ ]:
# Caso seja necessario recarregar, rodar esta celula
from kedro.framework.session import KedroSession

# Recarregar o contexto e o catálogo
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

# Acessa os parâmetros
params = context.params

catalog.list()

In [ ]:
with KedroSession.create(env="local", project_path=project_path) as session:
    session.run(pipeline_name="webscraping")

In [6]:
import logging

def setup_logger(logger_name:str, 
                 log_folder: str = None) -> logging.Logger:
    """
    Configura o logger para salvar os logs em um arquivo ou exibi-los no terminal.
    
    Args:
        log_folder (str, optional): Caminho do arquivo onde os logs serão salvos. 
        Se for None, o logger exibirá as informações no terminal.
    
    Returns:
        logging.Logger: Logger configurado para salvar logs em arquivo ou exibi-los no terminal.
    """
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)

    # Desativa a propagação do logger para o logger raiz
    logger.propagate = False

    # Verifica se já existem handlers para evitar duplicação de logs
    if not logger.hasHandlers():
        if log_folder:
            # Criar um handler para salvar o log em arquivo
            file_handler = logging.FileHandler(log_folder, mode='w', encoding='utf-8')
            file_handler.setLevel(logging.INFO)

            # Criar um formato para o log
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(formatter)

            # Adicionar o handler ao logger
            logger.addHandler(file_handler)
        else:
            # Criar um StreamHandler para exibir os logs no terminal
            console_handler = logging.StreamHandler()
            console_handler.setLevel(logging.INFO)

            # Criar um formato para o log no console
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            console_handler.setFormatter(formatter)

            # Adicionar o handler ao logger
            logger.addHandler(console_handler)

    return logger

In [ ]:
import os
import requests
import zipfile

def get_deck_zip_from_web(        
        project_path: str,
        zip_url: str,  
        zip_folder: str) -> None:
    """
    Baixa um arquivo ZIP de um URL, o descompacta em uma pasta temporária e salva os arquivos JSON
    no subdiretório 'decks_json' dentro da pasta especificada.

    Args:
        project_path (str): Caminho base do projeto onde o zip_folder será concatenado.
        zip_url (str): URL do arquivo ZIP a ser baixado.
        zip_folder (str): Caminho relativo dentro do project_path onde o arquivo ZIP será salvo.

    Returns:
        None: A função salva os arquivos JSON na pasta especificada e não retorna nada.
    """
    # Configura o logger geral com o nome "get_deck_zip_logger"
    logger = setup_logger("get_deck_zip_logger")

    # Concatenar o caminho completo de zip_folder com project_path (que é a raiz)
    full_output_path = os.path.join(project_path, zip_folder)
    
    # Criar a pasta raiz se ela não existir
    os.makedirs(full_output_path, exist_ok=True)

    # Criar o subdiretório 'decks_json' dentro da pasta especificada
    decks_json_path = os.path.join(full_output_path, 'decks_json')
    os.makedirs(decks_json_path, exist_ok=True)

    # Função para baixar o arquivo zip
    def download_file(url, folder):
        local_filename = os.path.join(folder, url.split("/")[-1])
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        logger.info(f"Zip dos decklists baixado com sucesso em: {local_filename}")
        return local_filename

    # Baixar o arquivo zip
    zip_file_path = download_file(zip_url, full_output_path)

    # Descompactar o arquivo zip e salvar os arquivos JSON no subdiretório 'decks_json'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.endswith('.json'):
                # Definir o caminho de destino do arquivo extraído no subdiretório 'decks_json'
                output_file_path = os.path.join(decks_json_path, file_name)
                
                # Ler e salvar o arquivo JSON diretamente no caminho especificado
                with zip_ref.open(file_name) as json_file, open(output_file_path, "wb") as out_file:
                    out_file.write(json_file.read())
    
    logger.info(f"Arquivos JSON extraídos com sucesso em: {decks_json_path}")

    # Remover o arquivo zip após extração
    os.remove(zip_file_path)
    logger.info(f"Arquivo ZIP removido após extração!")

    # Remover o handler para evitar problemas futuros
    for handler in logger.handlers:
        handler.close()
        logger.removeHandler(handler)


project_path = catalog.load("params:global.user.project_path")
zip_url = catalog.load("params:preprocessing.webscraper.zip_url")
zip_folder = catalog.load("params:preprocessing.webscraper.zip_folder")

get_deck_zip_from_web(project_path,zip_url, zip_folder)

In [ ]:
def pp_decks_from_json_files(
        decks_json_partitioned: dict, 
        deck_cards: int, 
        log_folder: str) -> dict:
    """
    Processa todos os decks JSON fornecidos pelo PartitionedDataSet e salva no formato .txt.

    Faz isso desde que contenham pelo menos o número mínimo de cartas definido em deck_cards (params).
    O log dos decks processados é salvo em um arquivo no log_folder (params).

    Args:
        decks_json_partitioned (dict): Dicionário de decks carregados de arquivos JSON através de PartitionedDataSet.
        deck_cards (int): Número mínimo de cartas no mainBoard para que o deck seja processado.
        log_folder (str): Caminho do arquivo onde os logs serão salvos.

    Returns:
        dict: Dicionário de decks processados, onde as chaves são os nomes dos arquivos e os valores são as decklists.
    """
    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_selection_log.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Chama a função para configurar o logger
    logger = setup_logger("pp_decks_from_json_files", log_filepath)

    # Função para extrair o nome do deck diretamente do JSON
    def get_deck_name(data):
        return data['data'].get('name', 'Unknown Deck')

    # Função para contar o número de cartas no mainBoard
    def count_mainboard_cards(data):
        return sum(card['count'] for card in data['data']['mainBoard'])

    # Função para gerar a decklist formatada
    def generate_decklist(data):
        decklist = []
        deck_name = get_deck_name(data)
        decklist.append("About")
        decklist.append(f"Name {deck_name}")
        decklist.append("\nDeck")

        for card in data['data']['mainBoard']:
            name = card['name']
            count = card['count']
            decklist.append(f"{count} {name}")

        return decklist

    # Dicionário de saída para armazenar as decklists processadas
    processed_decks = {}

    # Percorrer os arquivos JSON no dicionário fornecido pelo PartitionedDataSet
    for file_name, dataset in decks_json_partitioned.items():
        # Carregar os dados chamando o método `load()` do dataset
        data = dataset()

        # Verificar se o deck tem pelo menos deck_cards no mainboard
        if count_mainboard_cards(data) >= deck_cards:
            
            # Gerar a decklist
            decklist = generate_decklist(data)

            # Definir o nome do arquivo de saída .txt com base no nome do deck
            deck_name = get_deck_name(data)
            output_file_name = f"{deck_name.replace(' ', '_')}.txt"

            # Salvar no dicionário de decks processados
            processed_decks[output_file_name] = "\n".join(decklist)

            logger.info(f"Decklist {output_file_name} gerada com sucesso!")
        else:
            logger.info(f"Deck {file_name} ignorado (menos de {deck_cards} cartas no mainBoard).")

    # Remover o handler para evitar problemas futuros
    for handler in logger.handlers:
        handler.close()
        logger.removeHandler(handler)

    return processed_decks

decks_json_partitioned = catalog.load("decks_json_partitioned")
deck_cards = catalog.load("params:preprocessing.webscraper.deck_cards")
log_folder = catalog.load("params:preprocessing.webscraper.log_folder")

txt_decks_dict = pp_decks_from_json_files(decks_json_partitioned, deck_cards, log_folder)
txt_decks_dict

In [16]:
# Criando a pasta artificialmente aqui (para evitar rodar o pipeline)
output_folder = project_path + '/data/01_raw/decks_txt'

# Criar a pasta caso ela não exista
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre o dicionário e salvar cada deck como um arquivo .txt
for file_name, deck_content in txt_decks_dict.items():
    # Caminho completo do arquivo de saída
    output_file_path = os.path.join(output_folder, file_name)
    
    # Escrever o conteúdo do deck no arquivo .txt
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(deck_content)

In [ ]:
import random

def sample_decks(
        decks_txt_partitioned: dict, 
        sample_size: float, 
        log_folder: str) -> dict:
    """
    Amostra os decks no formato .txt com base no sample_size e retorna um dicionário de paths.

    Args:
        decks_txt_partitioned (dict): Dicionário de decks onde as chaves são nomes de arquivos e os valores são funções que retornam o conteúdo do deck.
        sample_size (float): A fração da população original que será usada para amostragem (valor entre 0 e 1).
        log_folder (str): Caminho da pasta para salvar os logs.

    Returns:
        dict: Dicionário com os caminhos dos decks amostrados.
    """

    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_sampling_log.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configurar o logger para salvar no arquivo .txt
    logger = setup_logger('decks_sampling_logger', log_filepath)

    # Número total de decks disponíveis
    total_decks = len(decks_txt_partitioned)
    
    # Calcular o tamanho da amostra
    target_sample_size = int(total_decks * sample_size)

    # Amostra aleatória da população
    logger.info(f"Amostrando {target_sample_size} decks de um total de {total_decks}.")
    sampled_keys = random.sample(list(decks_txt_partitioned.keys()), target_sample_size)
    
    # Criar dicionário contendo os paths dos decks amostrados
    sampled_decks = {key: os.path.join('data/01_raw/decks_txt', key) for key in sampled_keys}

    # Logar os decks escolhidos
    logger.info(f"Decks amostrados: {', '.join(sampled_keys)}")
    logger.info(f"Amostragem completa. {len(sampled_decks)} decks selecionados.")

    return sampled_decks


decks_txt = catalog.load("decks_txt_partitioned")
sample_size = catalog.load("params:preprocessing.webscraper.sample_size_ratio")
log_folder = catalog.load("params:preprocessing.webscraper.log_folder")

sample_decks_dict = sample_decks(decks_txt, sample_size, log_folder)
catalog.save("sampled_decks",sample_decks_dict)